## 写在前面

一直在用 pandas 来做数据处理和分析方面的工作，真心感到 pandas 的强大和宜用。但是我一直有个感觉，自己没有很好的理解和掌握 pandas 里的用法和一些基础概念，所以就有了这几篇博文[练习]。目的是总结一些 pandas，dataframe 的基础概念，记录一些我常用到的基本 API 的使用，详细记录一些高级 API 的用途，比如 groupby 和 pivot 等。

这是第一篇，记录下一半都有哪些常用的构建 DataFrame 的方法。

In [28]:
# built-in packages
import os
import sys
import json
import time
import datetime

# third-party packages
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn
import mpld3

## 1. 构建 dataframe

事实上，有很多种方式来构建一个 dataframe，不过总结下来其实就三大类： 

- 通过 pd.DataFrame 构造函数
- 通过 IO tools 来构造，当前 pandas 所支持的io tools已经足够多了，移步这里[io tools](http://pandas.pydata.org/pandas-docs/stable/io.html)
- 通过其他 dataframe 生成新的 dataframe

因为我用得最多的都是读取 csv，xls，sql，这几种方式，所以我这里就记录这几种方式，关键点是记录用这些方法时的一些tricks和细节。

### 1.1 pd.read_csv, pd.read_excel


pd.read_csv 有很多参数，但是我觉得最常用的参赛［其实我觉得很多参赛都很有用的，但我目前常用的就下面几个，哈哈］有下面几个：

- filepath_or_buffer: 指定文件路径，注意，只要可以生成一个有效的文件对象都可以。所以这个参数可以是：
    + 本地文件路径，最常用，大多数人也只知道这样用
    + StringIO, cStringIO 对象
    + URL: 可以是 http，ftp，s3和file这几种，牛气了吧，说说有多少次你是先把文件下载下来再load到dataframe里的。不过很多时候先把文件download下来也是一个不错的选择，特别是当文件很大，而且你会多次load那个文件的时候。
- sep：指定csv文件的分隔符，一般默认都是","了，当然有的csv是以tab分割的，这时候就要显式指定分隔符了；
- names：指定你的csv文件每一列的列名，当你的csv文件的第一行也是数据部分的话就要用到这个参数了，否则生成的dataframe会吧第一行当作dataframe的column name；


pd.read_excel 有一些参赛和 read_csv 很像，其实我觉得excel 和csv 最大的不同可能就是excel 有sheet 的这个概念吧：

- io：和csv的filepath_or_buffer差不多；
- sheetname: excel 特有的，如果指定一个，就读取特定的sheet；如果指定多个，就读取多个sheet，然后返回一个字典，key是sheetname，value是对应的dataframe；默认读取第一个sheet的data；

In [29]:
# example read csv
url = 'https://raw.githubusercontent.com/litaotao/data-science/master/books_courses/Python%20for%20Data%20Analysis/ch02/names/yob1880.txt'
pd.read_csv(url).head(2)

,Mary,F,7065
0,Anna,F,2604
1,Emma,F,2003


In [30]:
pd.read_csv(url, names=['name', 'gender', 'births']).head(2)

,name,gender,births
0,Mary,F,7065
1,Anna,F,2604


### 1.2 pd.read_sql

pd.read_sql 是 pd.read_sql_table 和 pd.read_sql_query 的上层接口，调用read_sql的结果最后都是通过read_sql_table 或 read_sql_query中一个函数执行的结果。这个函数我觉得有一个参赛是最值得提的：

- chunksize：当数据量很大的时候，都期望把数据分片返回，但是需要注意的是，这里的chunksize虽然是用户手动指定的，但数据库查询是否支持chunk返回是由你的数据库连接决定的。也就是说，当你的数据库连接不支持chunk返回数据的时候，即使你指定了chunksize也没有用。

### 1.3 pd.DataFrame 构造函数

我很少用到构造函数来生成dataframe，但在写示例的时候就会经常用到它了。

- data: 任何可遍历的对象：
    + numpy 的ndarray对象，一半都是二维对象，否则代码写起来就不好看了；
    + dict，key是生成的dataframe的column name，注意如果是通过字典来构建dataframe的话，需要保证每个value的长度都一样；可以手动指定index，默认是从0开始的数字；index可以是字符串哦；
    + list-like object: 可遍历的对象，和numpy的ndarray优点像，具体看例子就知道了；

In [31]:
# from dict
data = {'A': range(0, 5), 'B': range(10, 15, 1)}
pd.DataFrame(data, index=range(100, 105, 1))

,A,B
100,0,10
101,1,11
102,2,12
103,3,13
104,4,14


In [32]:
# from list like object
pd.DataFrame([1,2,3])

,0
0,1
1,2
2,3


In [33]:
pd.DataFrame([[1,2,3]])

,0,1,2
0,1,2,3


In [34]:
pd.DataFrame([[1,2,3], [1,2,3]])

,0,1,2
0,1,2,3
1,1,2,3


In [35]:
pd.DataFrame([[[1,2,3], [1,2,3]]])

,0,1
0,"[1, 2, 3]","[1, 2, 3]"


In [36]:
# 如果还想活命的话，最好不要这样写，免得看代码的人忍不住来干掉你~
pd.DataFrame([[[[1,2,3], [1,2,3]]]])

,0
0,"[[1, 2, 3], [1, 2, 3]]"


## 参考资源

- [10 Minutes to pandas](http://pandas.pydata.org/pandas-docs/stable/10min.html)
- [pandas cookbook](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook)